In [15]:
import requests
from bs4 import BeautifulSoup
import cPickle as pickle
import time
from test import *
import random

In [16]:
# http://lanl.arxiv.org/list/math.NT/1412?show=250
dates = []
for year in ['14','13','12']:
    for i in range(12,0, -1):
        if len(str(i)) == 1:
            dates.append(year+'0'+str(i))
        else:
            dates.append(year+str(i))
#good_dates = dates[3:]
print len(dates)

front = 'http://lanl.arxiv.org/list/math.NT/'
end = '?show=250'
NT_urls = [front+x+end for x in dates]

print len(NT_urls)

NT_names = ['NT_'+date+'.pkl' for date in dates]

NT_names_new = ['NT_'+date+'_new.pkl' for date in dates]

small_NT_urls = NT_urls[:9]
small_NT_names = NT_names_new[:9]

print len(NT_names)

36
36
36


In [18]:
NT_urls[8]

'http://lanl.arxiv.org/list/math.NT/1404?show=250'

In [19]:
def get_text2(url):
    '''Input a URL from the arxiv (page of a list of papers), return a list of
    parsed articles (list of strings)
    '''

    #session = requesocks.session()

# #Use Tor for both HTTP and HTTPS
    #session.proxies = {'http': 'socks5://127.0.0.1:9150', 'https': 'socks5://127.0.0.1:9150'}


#http://lanl.arxiv.org/pdf/1508.00039.pdf

    base_url = url
    r = requests.get(base_url, headers={'User-agent': 'Mozilla/5.0'})
    #print r.status_code
    #r = session.get(base_url, headers={'User-agent': 'Mozilla/5.0'})
    print "Status: ", r.status_code

    soup = BeautifulSoup(r.text, 'html.parser')
    pdfs = soup.findAll(title = 'Download PDF')
    links = [str(pdf).split()[1].strip('href="') for pdf in pdfs]
    urls = ['http://lanl.arxiv.org'+ link for link in links]
    titles = soup.findAll(class_="list-title")
    title_list = []
    for ix in range(len(titles)):
        title_list.append(titles[ix].text)
    url_title = zip(urls, title_list)
    
    articles = []
    failed = []
    good_urls = []
    pool = multiprocessing.Pool(processes=4)
    articles = pool.map(parsePDF2, url_title)
    
#     index = 0
#     for url in urls:
#         print index
#         try:
#             text = parsePDF(url)
#             articles.append(text)
#             title_list.append(titles[index].text)
#             good_urls.append(url)
#             print "Articles: ", len(articles)
#             print "Titles: ", len(title_list)
#             x=random.randint(0,30)
#             print "Sleep now, {0} seconds".format(x)
#             time.sleep(x)
#             index += 1
#         except:
#             print 'unexpected error'
#             failed.append((url, index))
#             print "Num Failed: ", len(failed)
#             index += 1
    

#     for pair in failed: 
#         print pair[0]
#         try: 
#             articles.append(parsePDF(pair[0]))
#             title_list.append(titles[pair[1]].text)
#             good_urls.append(pair[0])
#             print "articles: ", len(articles)
#             print "titles: ", len(title_list)
#             x=random.randint(0,30)
#             print "Sleep now, {0} seconds".format(x)
#             time.sleep(random.randint(0,30))
#         except:
#             print 'unexpected error'
            
#     print 'Num failures: ', len(failed)
#     print 'Num articles: ', len(articles)
            
#    titles = soup.findAll(class_="list-title")
#    title_list = []
#    for ix in range(len(titles)):
#        title_list.append(titles[ix].text)
    print "Returned {0} articles".format(len(articles))
    print "Status: ", r.status_code
    return articles #, title_list, good_urls
    #return title_list, urls, links, pdfs

In [20]:
#test_url = 'http://lanl.arxiv.org/list/math.NT/1509?show=5'

In [21]:
# name = NT_names[0]
# with open(name, 'w') as f:
#     pickle.dump(stuff, f)

In [22]:
# for url, name in zip(small_NT, small_NT_names):
#     stuff = get_text2(url)
#     with open(name, 'w') as f:
#         pickle.dump(stuff, f)

In [23]:
def parsePDF2(url):

    # Open the url provided as an argument to the function and read the content
    try:
        open = urllib2.urlopen(Request(url[0])).read()

        # Cast to StringIO object
        from StringIO import StringIO
        memory_file = StringIO(open)

        # Create a PDF parser object associated with the StringIO object
        parser = PDFParser(memory_file)

        # Create a PDF document object that stores the document structure
        document = PDFDocument(parser)

        # Define parameters to the PDF device objet
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        laparams = LAParams()
        codec = 'utf-8'

        # Create a PDF device object
        device = TextConverter(rsrcmgr, retstr, codec = codec, laparams = laparams)

        # Create a PDF interpreter object
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        # Process each page contained in the document
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
            data =  retstr.getvalue()
        sl = random.randint(0,30)
        #print "Time to sleep for {0} seconds".format(sl)
        time.sleep(sl)
        return data, url[0], url[1]

    except: 
        x = random.randint(5,15)
        #print "Didn't work, try again in {0} seconds".format(x)
        time.sleep(x)
        try:
            open = urllib2.urlopen(Request(url[0])).read()

            # Cast to StringIO object
            from StringIO import StringIO
            memory_file = StringIO(open)

            # Create a PDF parser object associated with the StringIO object
            parser = PDFParser(memory_file)

            # Create a PDF document object that stores the document structure
            document = PDFDocument(parser)

            # Define parameters to the PDF device objet
            rsrcmgr = PDFResourceManager()
            retstr = StringIO()
            laparams = LAParams()
            codec = 'utf-8'

            # Create a PDF device object
            device = TextConverter(rsrcmgr, retstr, codec = codec, laparams = laparams)

            # Create a PDF interpreter object
            interpreter = PDFPageInterpreter(rsrcmgr, device)

            # Process each page contained in the document
            for page in PDFPage.create_pages(document):
                interpreter.process_page(page)
                data =  retstr.getvalue()
            return data, url[0], url[1]
        except:
            #print "Sorry, can't get this paper"
            time.sleep(random.randint(5,10))
            

In [24]:
#%time arts = get_text2('http://lanl.arxiv.org/list/math.NT/1501?show=250')

In [25]:
small_NT_names[8]

'NT_1404_new.pkl'

In [26]:
# %%time
# arts = get_text2(small_NT_urls[8])
# good_type = 0
# bad_type = 0
# good_arts = []
# for ix in range(len(arts)):
#     if type(arts[ix]) == tuple:
#         good_type += 1
#         good_arts.append(arts[ix])
#     else:
#         bad_type += 1
# print "Good: ", good_type
# print "Good again: ", len(good_arts)
# print "Bad: ", bad_type
# with open(small_NT_names[8], 'w') as f:
#     pickle.dump(good_arts, f)

In [27]:
# text = []
# for article in good_arts:
#     text.append(article[0])

In [28]:
# good_type = 0
# bad_type = 0
# better_arts = []
# for ix in range(len(good_arts)):
#     if type(good_arts[ix][2]) == str:
#         good_type += 1
#         better_arts.append(good_arts[ix])
#     else:
#         bad_type += 1
# print good_type
# print len(better_arts)
# print bad_type

In [29]:
r = requests.get('http://lanl.arxiv.org/archive/math', headers={'User-agent': 'Mozilla/5.0'})
soup = BeautifulSoup(r.text, 'html.parser')
print r.status_code
match_math = re.findall(r'math\.\w\w', soup.text)
match_math = list(set(match_math))
print len(match_math)
print match_math

200
32
[u'math.NA', u'math.DG', u'math.HO', u'math.FA', u'math.DS', u'math.PR', u'math.LO', u'math.RA', u'math.NT', u'math.GR', u'math.CO', u'math.AG', u'math.GT', u'math.AC', u'math.IT', u'math.CA', u'math.AT', u'math.AP', u'math.CV', u'math.RT', u'math.CT', u'math.GN', u'math.GM', u'math.MG', u'math.OC', u'math.ST', u'math.OA', u'math.SP', u'math.MP', u'math.KT', u'math.QA', u'math.SG']


In [30]:
front = 'http://lanl.arxiv.org/list/'
end = '/1506?show=250'
math_urls = [front+x+end for x in match_math]
math_tags = [x[-2:] for x in match_math]
math_names = ['math_'+ x + '_June.pkl' for x in math_tags]
print math_names[0]
print math_urls[0]

math_NA_June.pkl
http://lanl.arxiv.org/list/math.NA/1506?show=250


In [31]:
def parsePDF3(url):

    # Open the url provided as an argument to the function and read the content
    try:
        open = urllib2.urlopen(Request(url[0])).read()

        # Cast to StringIO object
        from StringIO import StringIO
        memory_file = StringIO(open)

        # Create a PDF parser object associated with the StringIO object
        parser = PDFParser(memory_file)

        # Create a PDF document object that stores the document structure
        document = PDFDocument(parser)

        # Define parameters to the PDF device objet
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        laparams = LAParams()
        codec = 'utf-8'

        # Create a PDF device object
        device = TextConverter(rsrcmgr, retstr, codec = codec, laparams = laparams)

        # Create a PDF interpreter object
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        # Process each page contained in the document
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
            data =  retstr.getvalue()
        sl = random.randint(0,30)
        #print "Time to sleep for {0} seconds".format(sl)
        time.sleep(sl)
        return data, url[0], url[1], 'DG'

    except: 
        x = random.randint(5,15)
        #print "Didn't work, try again in {0} seconds".format(x)
        time.sleep(x)
        try:
            open = urllib2.urlopen(Request(url[0])).read()

            # Cast to StringIO object
            from StringIO import StringIO
            memory_file = StringIO(open)

            # Create a PDF parser object associated with the StringIO object
            parser = PDFParser(memory_file)

            # Create a PDF document object that stores the document structure
            document = PDFDocument(parser)

            # Define parameters to the PDF device objet
            rsrcmgr = PDFResourceManager()
            retstr = StringIO()
            laparams = LAParams()
            codec = 'utf-8'

            # Create a PDF device object
            device = TextConverter(rsrcmgr, retstr, codec = codec, laparams = laparams)

            # Create a PDF interpreter object
            interpreter = PDFPageInterpreter(rsrcmgr, device)

            # Process each page contained in the document
            for page in PDFPage.create_pages(document):
                interpreter.process_page(page)
                data =  retstr.getvalue()
            return data, url[0], url[1], label, 'HO'
        except:
            #print "Sorry, can't get this paper"
            time.sleep(random.randint(5,10))

In [32]:
def get_text3(url):
    '''Input a URL from the arxiv (page of a list of papers), return a list of
    parsed articles (list of strings)
    '''

    #session = requesocks.session()

# #Use Tor for both HTTP and HTTPS
    #session.proxies = {'http': 'socks5://127.0.0.1:9150', 'https': 'socks5://127.0.0.1:9150'}


#http://lanl.arxiv.org/pdf/1508.00039.pdf

    base_url = url
    r = requests.get(base_url, headers={'User-agent': 'Mozilla/5.0'})
    #print r.status_code
    #r = session.get(base_url, headers={'User-agent': 'Mozilla/5.0'})
    print "Status: ", r.status_code

    soup = BeautifulSoup(r.text, 'html.parser')
    pdfs = soup.findAll(title = 'Download PDF')
    links = [str(pdf).split()[1].strip('href="') for pdf in pdfs]
    urls = ['http://lanl.arxiv.org'+ link for link in links]
    titles = soup.findAll(class_="list-title")
    title_list = []
    for ix in range(len(titles)):
        title_list.append(titles[ix].text)
    url_title = zip(urls, title_list)
    
    articles = []
    failed = []
    good_urls = []
    pool = multiprocessing.Pool(processes=4)
    articles = pool.map(parsePDF3, url_title)
    
#     index = 0
#     for url in urls:
#         print index
#         try:
#             text = parsePDF(url)
#             articles.append(text)
#             title_list.append(titles[index].text)
#             good_urls.append(url)
#             print "Articles: ", len(articles)
#             print "Titles: ", len(title_list)
#             x=random.randint(0,30)
#             print "Sleep now, {0} seconds".format(x)
#             time.sleep(x)
#             index += 1
#         except:
#             print 'unexpected error'
#             failed.append((url, index))
#             print "Num Failed: ", len(failed)
#             index += 1
    

#     for pair in failed: 
#         print pair[0]
#         try: 
#             articles.append(parsePDF(pair[0]))
#             title_list.append(titles[pair[1]].text)
#             good_urls.append(pair[0])
#             print "articles: ", len(articles)
#             print "titles: ", len(title_list)
#             x=random.randint(0,30)
#             print "Sleep now, {0} seconds".format(x)
#             time.sleep(random.randint(0,30))
#         except:
#             print 'unexpected error'
            
#     print 'Num failures: ', len(failed)
#     print 'Num articles: ', len(articles)
            
#    titles = soup.findAll(class_="list-title")
#    title_list = []
#    for ix in range(len(titles)):
#        title_list.append(titles[ix].text)
    print "Returned {0} articles".format(len(articles))
    print "Status: ", r.status_code
    return articles #, title_list, good_urls
    #return title_list, urls, links, pdfs

In [33]:
print math_tags[2]
print math_urls[2]
print math_names[2]

HO
http://lanl.arxiv.org/list/math.HO/1506?show=250
math_HO_June.pkl


In [ ]:
%%time
for url, name in zip(NT_urls[:3], NT_names[:3]):
    arts = get_text2(url)
    good_type = 0
    bad_type = 0
    good_arts = []
    for ix in range(len(arts)):
        if type(arts[ix]) == tuple:
            good_type += 1
            good_arts.append(arts[ix])
        else:
            bad_type += 1
    print "Good: ", good_type
    print "Good again: ", len(good_arts)
    print "Bad: ", bad_type
    for ix in range(len(good_arts)):
        good_arts[ix] += ('NT',)
    with open(name, 'w') as f:
        pickle.dump(good_arts, f)

Status:  200
Returned 180 articles
Status:  200
Good:  94
Good again:  94
Bad:  86
Status:  200


In [23]:
math_tags[17]

u'AP'

In [84]:
good_arts[0][3]

'DG'